In [21]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [22]:
API_key = 'B51wnmtnL30svioyDrNUrhl4h7We2HR8i-DQgKUNPsbVsiruZAItOeM6IHC8e5Y72G-d9TjHZFQHtKJjBrSCWCF0gLTvDlfr9u7bKce5jClTMT4ua-m0dtzxI6q1XXYx'

In [23]:
from yelpapi import YelpAPI
import argparse
from pprint import pprint
#see help code https://github.com/gfairchild/yelpapi/blob/master/examples/examples.py

In [24]:
import requests
import json
headers = {'Authorization': 'Bearer %s' % API_key}

In [25]:
yelp_api = YelpAPI(API_key)

In [6]:
day = int(input("Hello, what's the day of scrapping (day count start at 0) ?")) #day of scrapping

Hello, what's the day of scrapping (day count start at 0) ?2


In [7]:
if day==0: #If this is the first day of scrapping, we open the clean dataset
    data = pd.read_csv('data/chicago-food-inspections/food-inspections_clean.csv')
else: #Else, we open the previously scrapped dataset
    data = pd.read_csv('data/chicago-food-inspections/food-inspections_scrapped'+str(day-1)+'.csv')

In [27]:
data = pd.read_csv("data/chicago-food-inspections/food-inspections_scrapped"+str(day)+"_stopped_9874.csv")

In [28]:
data.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,License #,DBA Name,Facility Type,Risk,Address,City,State,Zip,Latitude,Longitude,Inspection Date,Inspection Type,Results,Yelp name,Yelp review,Yelp review count,Yelp rating
0,0,0,0,2.0,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,CHICAGO,IL,60606.0,41.880757158647214,-87.6347092983425,"['2018-02-13T00:00:00.000', '2017-05-12T00:00:...","['Canvass', 'Canvass', 'Short Form Complaint',...","['Pass w/ Conditions', 'Pass', 'Pass w/ Condit...",Cosi,"[4, 1, 5]",3.0,3.333333
1,1,1,1,9.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,41.880395838259616,-87.62450172159464,"['2019-08-09T00:00:00.000', '2018-06-19T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass w/ Conditions', 'Pass', 'Fail', 'Pass',...",NaN,NaN,NaN,NaN
2,2,2,2,40.0,COSI,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,41.886567370886944,-87.62438467059714,"['2018-09-14T00:00:00.000', '2018-03-27T00:00:...","['Complaint', 'Canvass', 'Complaint Re-Inspect...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail',...",Cosi,"[4, 4, 2]",3.0,3.333333
3,3,3,3,62.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,41.88331785985083,-87.63476909461744,"['2018-02-20T00:00:00.000', '2017-02-09T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass', 'Pass', 'Fail', 'Pass w/ Conditions',...",NaN,NaN,NaN,NaN
4,4,4,4,85.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),55 E GRAND AVE,CHICAGO,IL,60611.0,41.891590741083505,-87.62586713724458,"['2012-03-28T00:00:00.000', '2011-05-10T00:00:...","['Canvass', 'Canvass', 'Canvass Re-Inspection'...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail']",Cosi Sandwich Bar,"[4, 3, 3]",3.0,3.333333


Note: We have a bit more than 20'000 rows in our clean dataset, as Yelp API restrict us to 5'000 queries per day, we divided the number of rows by 5, and scrapped the whole dataset in 5 days.

In [20]:
n_days = 5 #Total number of days required
scrapping_per_day = int(data.shape[0]/n_days) #Number of rows to scrapp
if day==5:
    scrapping_per_day = scrapping_per_day + 1 #As the scrapping number is not an interger, we need to add 1 to obtain a complete scrapping

In [21]:
start = scrapping_per_day*day #Starting row for the scrapping
end = start + scrapping_per_day #Ending row for the scrapping
if start == 0: #If this is the first time that we scrapped the data, we need to create the new columns, else, they are already there
    data['Yelp name']=np.NaN
    data['Yelp review']=np.NaN
    data['Yelp review count']=np.NaN
    data['Yelp rating']=np.NaN
data.head()

,Unnamed: 0,License #,DBA Name,Facility Type,Risk,Address,City,State,Zip,Latitude,Longitude,Inspection Date,Inspection Type,Results,Yelp name,Yelp review,Yelp review count,Yelp rating
0,0,2.0,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,CHICAGO,IL,60606.0,41.880757158647214,-87.6347092983425,"['2018-02-13T00:00:00.000', '2017-05-12T00:00:...","['Canvass', 'Canvass', 'Short Form Complaint',...","['Pass w/ Conditions', 'Pass', 'Pass w/ Condit...",Cosi,"[4, 1, 5]",3.0,3.333333
1,1,9.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,41.880395838259616,-87.62450172159464,"['2019-08-09T00:00:00.000', '2018-06-19T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass w/ Conditions', 'Pass', 'Fail', 'Pass',...",NaN,NaN,NaN,NaN
2,2,40.0,COSI,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,41.886567370886944,-87.62438467059714,"['2018-09-14T00:00:00.000', '2018-03-27T00:00:...","['Complaint', 'Canvass', 'Complaint Re-Inspect...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail',...",Cosi,"[4, 4, 2]",3.0,3.333333
3,3,62.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,41.88331785985083,-87.63476909461744,"['2018-02-20T00:00:00.000', '2017-02-09T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass', 'Pass', 'Fail', 'Pass w/ Conditions',...",NaN,NaN,NaN,NaN
4,4,85.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),55 E GRAND AVE,CHICAGO,IL,60611.0,41.891590741083505,-87.62586713724458,"['2012-03-28T00:00:00.000', '2011-05-10T00:00:...","['Canvass', 'Canvass', 'Canvass Re-Inspection'...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail']",Cosi Sandwich Bar,"[4, 3, 3]",3.0,3.333333


In [31]:
for i in range(9874+2500,9874+3000):
    temp = data.iloc[i]
    response = yelp_api.business_match_query(name=temp['DBA Name'],
                                             address1=temp['Address'],
                                             city=temp['City'],
                                             state=temp['State'],
                                             country='US') #['businesses'][0]
    
    if len(response['businesses'])>0:
        data['Yelp name'].iloc[i] = response['businesses'][0]['name']
        id_ =  response['businesses'][0]['id']
        url = "https://api.yelp.com/v3/businesses/" + id_ + "/reviews"
        req = requests.get(url, headers=headers)
        #print('the status code is {}'.format(req.status_code))
        parsed = json.loads(req.text)
        rates = [parsed['reviews'][i]['rating'] for i in range(len(parsed['reviews']))]
        data['Yelp rating'].iloc[i] = np.mean(rates)
        data['Yelp review count'].iloc[i] = len(rates)
        data['Yelp review'].iloc[i] = str(rates)
    #print(temp['DBA Name'])
    #pprint(response)

YelpAPIError: ACCESS_LIMIT_REACHED: You've reached the access limit for this client. See instructions for requesting a higher access limit at https://www.yelp.com/developers/documentation/v3/rate_limiting

In [49]:
print("Among the rows", start,"and", end,"we scrapped",scrapping_per_day-np.count_nonzero(pd.isnull(data['Yelp name'][start:end])),"restaurants.")

Among the rows 4077 and 8154 we scrapped 2144 restaurants.


In [44]:
#Each day, we save the newly scrapped dataframe to a csv (so if an error occur at one point, we don't loose all the information from the previous days)
data.to_csv("data/chicago-food-inspections/food-inspections_scrapped"+str(day)+"_stopped_12743.csv")

In [44]:
day

1

In [32]:
print("we stopped at row",i)

we stopped at row 12743


In [30]:
end

8154

In [52]:
data.head(50)

,Unnamed: 0,License #,DBA Name,Facility Type,Risk,Address,City,State,Zip,Latitude,Longitude,Inspection Date,Inspection Type,Results,Yelp name,Yelp review,Yelp review count,Yelp rating
0,0,2.0,COSI,Restaurant,Risk 1 (High),230 W MONROE ST,CHICAGO,IL,60606.0,41.880757158647214,-87.6347092983425,"['2018-02-13T00:00:00.000', '2017-05-12T00:00:...","['Canvass', 'Canvass', 'Short Form Complaint',...","['Pass w/ Conditions', 'Pass', 'Pass w/ Condit...",Cosi,"[4, 1, 5]",3.0,3.333333
1,1,9.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),116 S MICHIGAN AVE,CHICAGO,IL,60603.0,41.880395838259616,-87.62450172159464,"['2019-08-09T00:00:00.000', '2018-06-19T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass w/ Conditions', 'Pass', 'Fail', 'Pass',...",NaN,NaN,NaN,NaN
2,2,40.0,COSI,Restaurant,Risk 1 (High),233 N MICHIGAN AVE,CHICAGO,IL,60601.0,41.886567370886944,-87.62438467059714,"['2018-09-14T00:00:00.000', '2018-03-27T00:00:...","['Complaint', 'Canvass', 'Complaint Re-Inspect...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail',...",Cosi,"[4, 4, 2]",3.0,3.333333
3,3,62.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),230 W WASHINGTON ST,CHICAGO,IL,60606.0,41.88331785985083,-87.63476909461744,"['2018-02-20T00:00:00.000', '2017-02-09T00:00:...","['Canvass', 'Canvass Re-Inspection', 'Canvass'...","['Pass', 'Pass', 'Fail', 'Pass w/ Conditions',...",NaN,NaN,NaN,NaN
4,4,85.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),55 E GRAND AVE,CHICAGO,IL,60611.0,41.891590741083505,-87.62586713724458,"['2012-03-28T00:00:00.000', '2011-05-10T00:00:...","['Canvass', 'Canvass', 'Canvass Re-Inspection'...","['Pass w/ Conditions', 'Pass', 'Pass', 'Fail']",Cosi Sandwich Bar,"[4, 3, 3]",3.0,3.333333
5,5,99.0,XANDO COFFEE & BAR / COSI SANDWICH BAR,Restaurant,Risk 1 (High),203 N LA SALLE ST,CHICAGO,IL,60601.0,41.88582204785303,-87.63230373508569,"['2016-04-06T00:00:00.000', '2015-12-08T00:00:...","['Canvass', 'Complaint', 'Canvass', 'Canvass',...","['Pass', 'Pass w/ Conditions', 'Pass', 'Pass',...",Cosi,"[3, 4, 1]",3.0,2.666667
6,6,115.0,JOHN SCHALLER,Restaurant,Risk 1 (High),3714 S HALSTED ST,CHICAGO,IL,60609.0,41.82718501563474,-87.6461704563508,"['2018-06-05T00:00:00.000', '2017-11-29T00:00:...","['Canvass', 'Canvass', 'Canvass', 'Canvass', '...","['No Entry', 'No Entry', 'No Entry', 'Pass', '...",Schaller's Pump,"[5, 5, 4]",3.0,4.666667
7,7,126.0,"CAL'S 400 LIQUORS, INC.",Restaurant,Risk 3 (Low),400 S WELLS ST,CHICAGO,IL,60607.0,41.876784831445576,-87.63383891021036,['2010-12-09T00:00:00.000'],['Canvass'],['Pass'],Cal's 400 Liquors,"[3, 5, 5]",3.0,4.333333
8,8,139.0,KILDARE LIQUORS,Restaurant,Risk 1 (High),4300 W LAWRENCE AVE,CHICAGO,IL,60630.0,41.968267042285326,-87.73549834843345,"['2010-01-26T00:00:00.000', '2010-01-19T00:00:...","['Canvass Re-Inspection', 'Canvass']","['Pass', 'Fail']",Kildare's Restaurant and Liquors,"[5, 4, 3]",3.0,4.000000
9,9,149.0,FOX'S BEVERLY PUB,Restaurant,Risk 1 (High),9956 S WESTERN AVE,CHICAGO,IL,60643.0,41.712012133229095,-87.68193540843004,"['2019-02-14T00:00:00.000', '2019-02-05T00:00:...","['Canvass Re-Inspection', 'Canvass', 'Canvass'...","['Pass w/ Conditions', 'Fail', 'Pass w/ Condit...",Fox's,"[4, 1, 5]",3.0,3.333333


In [42]:
data.iloc[i]

Unnamed: 0                                                        6911
License #                                                  1.98118e+06
DBA Name                                                    KING B.B.Q
Facility Type                                               Restaurant
Risk                                                     Risk 1 (High)
Address                                            6129 S Ashland AVE 
City                                                           CHICAGO
State                                                               IL
Zip                                                            60636.0
Latitude                                             41.78223235902597
Longitude                                           -87.66413997857381
Inspection Date      ['2013-04-08T00:00:00.000', '2011-01-19T00:00:...
Inspection Type                                 ['Canvass', 'Canvass']
Results                                 ['Pass', 'Pass w/ Conditions']
Yelp n

In [42]:
print("Among the rows", 0,"and", i,"we scrapped",i-np.count_nonzero(pd.isnull(data['Yelp name'][0:i])),"restaurants.", ((i-np.count_nonzero(pd.isnull(data['Yelp name'][0:i])))*100/i))

Among the rows 0 and 12743 we scrapped 9522 restaurants. 74.72337754061053
